# VectorDB Question Ansering with Sources

This notebook goes over how to do question-answering with sources over a vector database. It does this by using the `VectorDBQAWithSourcesChain`, which does the lookup of the documents from a vector database. 

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS

In [3]:
with open('../../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [4]:
docsearch = FAISS.from_texts(texts, embeddings)

In [5]:
# Add in a fake source information
for i, d in enumerate(docsearch.docstore._dict.values()):
    d.metadata = {'source': f"{i}-pl"}

In [3]:
from langchain.chains import VectorDBQAWithSourcesChain

In [8]:
from langchain import OpenAI

chain = VectorDBQAWithSourcesChain.from_llm(OpenAI(temperature=0), vectorstore=docsearch)

In [9]:
chain({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)

{'answer': ' The president thanked Justice Breyer for his service.',
 'sources': '30-pl'}